In [9]:
# 魔术指令，自动加载模块
%load_ext autoreload
%autoreload 2
import os
os.chdir("/home/beihang/xihu/HZTourism/FlowPred-dev")
import sys
sys.path.append("/home/beihang/xihu/HZTourism/FlowPred-dev")
import pandas as pd


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
his_hour=24
pred_hour=6
spot_id = 14100

# 配置项
exper_name = f"wheather_enhence_his{his_hour}h-pred{pred_hour}h"
exper_dir = f"exper/{exper_name}"
exper_data_dir = f"exper_data/{exper_name}"
os.makedirs(exper_data_dir, exist_ok=True)

raw_dir = f"{exper_data_dir}/raw/{spot_id}"
proc_dir = f"{exper_data_dir}/proc/{spot_id}"
train_dir = f"{exper_data_dir}/train/{spot_id}"
test_dir = f"{exper_data_dir}/test/{spot_id}"
res_dir = f"{exper_data_dir}/res/{spot_id}"


os.makedirs(raw_dir, exist_ok=True)
os.makedirs(proc_dir, exist_ok=True)
os.makedirs(train_dir, exist_ok=True)   
os.makedirs(test_dir, exist_ok=True)
os.makedirs(res_dir, exist_ok=True)


In [11]:
from src.utils.utils_data import get_spot_config

freq, his_len, pred_len = get_spot_config(spot_id, his_hour, pred_hour)


In [12]:
# 根据景点加载训练数据
from src.utils.utils_eva_db import save_csv_from_db

s_time = "2024-01-01"
e_time = "2024-12-31"
if e_time is None:
    file_base_name = f"{spot_id}_{s_time}"
    train_raw_data_file=f"{raw_dir}/{file_base_name}.csv"
else:
    file_base_name = f"{spot_id}_{s_time}_{e_time}"
    train_raw_data_file=f"{raw_dir}/{file_base_name}.csv"
save_csv_from_db(
    spot_id=spot_id,
    s_time=f"{s_time} 00:00:00",
    e_time= f"{e_time} 23:59:59",
    output_csv_file=train_raw_data_file,
)

数据库连接成功！
正在从表 'mobile_signaling_tourists_num' 中查询数据，景点ID: 14100，时间范围: 2024-01-01 00:00:00 至 2024-12-31 23:59:59...
输出表头: ['spot_id', 'kpi_time', 'kpi_value']
CSV 文件表头已写入。
成功！筛选后的数据已导出到文件 'exper_data/wheather_enhence_his24h-pred6h/raw/14100/14100_2024-01-01_2024-12-31.csv'，共 91007 条记录。
数据库连接已关闭。


In [13]:
# 数据预处理

df = pd.read_csv(train_raw_data_file)
# 按kpi_time列转换为datetime格式
df['kpi_time'] = pd.to_datetime(df['kpi_time'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
# 按kpi_time去重
df = df.drop_duplicates(subset=['kpi_time'])
# 按kpi_time排序
df = df.sort_values(by='kpi_time')
# 数据预处理，分景点
if spot_id in [14210,14211,14212,14213]:
    pass
elif spot_id in [14207,14209]:
    from src.utils.utils_data import fill_missing_value_singlespot_30s,preprocess_for_koopman_30s_moderate
    df_proc = fill_missing_value_singlespot_30s(spot_id, df)
    # 添加Koopman专用预处理
    # df_proc = preprocess_for_koopman_30s_moderate(df_proc, spot_id)
elif spot_id in [14208]:
    pass
else:
    from src.utils.utils_data import fill_missing_value_singlespot_day
    df_proc = fill_missing_value_singlespot_day(df, freq=freq)

df_proc.to_csv(f"{proc_dir}/{file_base_name}_proc.csv", index=False)

In [14]:
# 数据处理为模型输入形式

from src.pattern.pattern_train import get_group_annotation, save_mode_data
df_proc = pd.read_csv(f"{proc_dir}/{file_base_name}_proc.csv")
save_base_dir = train_dir
os.makedirs(save_base_dir, exist_ok=True)
groups_mode_0, groups_mode_1 = get_group_annotation(his_len=his_len,pred_len=pred_len, df=df_proc, time_interval=freq)
save_mode_data(
    groups_mode=groups_mode_0,
    mode=0,
    data_basepath=save_base_dir,
)
save_mode_data(
    groups_mode=groups_mode_1,
    mode=1,
    data_basepath=save_base_dir,
)

Starting group annotation...
Getting continuous groups...


Building continuous groups: 100%|██████████| 94464/94464 [00:04<00:00, 19012.46row/s]


Found 2 continuous groups
Processing groups (his_len=288, pred_len=72)...


Processing continuous groups: 100%|██████████| 2/2 [00:55<00:00, 27.55s/group]


Generated 63 holiday batches and 126 workday batches
Final merging and deduplicating...


...

Final holiday groups: 45
Final workday groups: 47
Saving 45 groups to mode_0...


Saving mode_0 files: 100%|██████████| 45/45 [00:00<00:00, 316.34file/s]


Mode_0 data saved successfully!
Saving 47 groups to mode_1...


Saving mode_1 files: 100%|██████████| 47/47 [00:00<00:00, 222.55file/s]

Mode_1 data saved successfully!


In [15]:
# 20000到20025的列表
[20000, 20001, 20002, 20003, 20004, 20005, 20006, 20007, 20008, 20009, 20010, 20011, 20012, 20013, 20014, 20015, 20016, 20017, 20018, 20019, 20020, 20021, 20022, 20023, 20024, 20025]

[20000,
 20001,
 20002,
 20003,
 20004,
 20005,
 20006,
 20007,
 20008,
 20009,
 20010,
 20011,
 20012,
 20013,
 20014,
 20015,
 20016,
 20017,
 20018,
 20019,
 20020,
 20021,
 20022,
 20023,
 20024,
 20025]